<a href="https://colab.research.google.com/github/kushb2/Techno-Funda/blob/main/Snippets_Importing_libraries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# ==============================================================================
# PROJECT: INSTITUTIONAL SNIPER (v5.0 - DEEP DIVE EDITION)
# OBJECTIVE: Scan stocks & Visualize 20-Day History for Verification
# ==============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
import time

# Try importing nsepython
try:
    from nsepython import nse_quote
except ImportError:
    nse_quote = None
    print("⚠️ NSEPython not found. Please install: pip install nsepython")

# ==============================================================================
# 1. CONFIGURATION
# ==============================================================================
class Config:
    # --- YOUR WATCHLIST ---
    WATCHLIST = [

        "NETWEB.NS"
    ]

    # --- Analysis Thresholds ---
    MIN_ROE = 10.0
    MAX_DEBT = 1.5

    # --- The Sniper Scope ---
    VOL_LOOKBACK = 20
    Z_SCORE_LIMIT = 2.0
    DELIVERY_LIMIT = 40.0

    SIMULATE_NSE = False     # <--- LIVE MODE

# ==============================================================================
# 2. DATA ENGINE
# ==============================================================================
def fetch_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        # fast_info is faster for basic data
        info = stock.fast_info
        # But we need .info for debt/roe (this can be slow)
        fund_info = stock.info

        hist = stock.history(period="1y") # Need 1 year for 52W High
        if hist.empty: return None, None
        return fund_info, hist
    except:
        return None, None

def get_delivery(ticker):
    """Fetches REAL Delivery % from NSE."""
    if nse_quote is None: return 0.0

    sym = ticker.replace(".NS", "")
    try:
        data = nse_quote(sym, section="trade_info")
        if data and 'securityWiseDP' in data:
            delivery_data = data['securityWiseDP']
            return float(delivery_data.get('deliveryToTradedQuantity', 0))
        return 0.0
    except:
        return 0.0

# ==============================================================================
# 3. ANALYSIS LOGIC
# ==============================================================================
def analyze_fundamentals(info):
    warnings = []
    try:
        roe = info.get('returnOnEquity', 0)
        if roe: roe = roe * 100

        debt = info.get('debtToEquity', 0)
        if debt: debt = debt / 100

        if roe and roe < Config.MIN_ROE: warnings.append(f"Low ROE ({roe:.1f}%)")
        if debt and debt > Config.MAX_DEBT: warnings.append(f"High Debt ({debt:.1f})")
    except:
        return "⚠️ Data Missing"
    return ", ".join(warnings) if warnings else "✅ Safe"

def check_institutional_action(hist, ticker):
    try:
        # Vol Stats
        window = Config.VOL_LOOKBACK
        vol_mean = hist['Volume'].rolling(window=window).mean().iloc[-1]
        vol_std = hist['Volume'].rolling(window=window).std().iloc[-1]
        curr_vol = hist['Volume'].iloc[-1]

        if vol_std == 0: return "Wait", "Stable"
        z_score = (curr_vol - vol_mean) / vol_std

        deliv = get_delivery(ticker)

        is_big_vol = z_score > Config.Z_SCORE_LIMIT
        is_high_deliv = deliv > Config.DELIVERY_LIMIT
        msg = f"Z:{z_score:.1f}, Del:{deliv:.1f}%"

        # Candle Shape
        O = hist['Open'].iloc[-1]
        H = hist['High'].iloc[-1]
        L = hist['Low'].iloc[-1]
        C = hist['Close'].iloc[-1]

        body = abs(C - O)
        lower_wick = min(O, C) - L
        range_len = H - L

        if range_len == 0: return "Wait", "No Move"

        # Signals
        if is_big_vol and is_high_deliv:
            is_hammer = (lower_wick > body * 2) and (C > L + (range_len * 0.6))
            if is_hammer: return "🚀 BUY (Spring)", f"Rejection ({msg})"
            elif C > O:   return "🚀 BUY (Strong)", f"Momentum ({msg})"
            else:         return "⚠️ WATCH", f"Red Candle ({msg})"

        if is_big_vol and not is_high_deliv:
            return "⚠️ FAKE PUMP", f"Low Deliv ({msg})"

        return "Wait", f"Normal ({msg})"
    except:
        return "Err", "Calc Error"

def get_detailed_metrics(hist):
    """Calculates 52-Week and 20-Day Highs/Lows with Dates."""
    try:
        # 1. 52-Week High
        last_year = hist.tail(252)
        high_52 = last_year['High'].max()
        # idxmax gives the index (Date) of the max value
        date_52 = last_year['High'].idxmax().strftime('%Y-%m-%d')

        # 2. 20-Day High/Low
        last_20 = hist.tail(20)
        high_20 = last_20['High'].max()
        date_high_20 = last_20['High'].idxmax().strftime('%Y-%m-%d')

        low_20 = last_20['Low'].min()
        date_low_20 = last_20['Low'].idxmin().strftime('%Y-%m-%d')

        return {
            "52W High": f"{high_52:.0f} ({date_52})",
            "20D High": f"{high_20:.0f} ({date_high_20})",
            "20D Low":  f"{low_20:.0f} ({date_low_20})"
        }
    except:
        return {"52W High": "-", "20D High": "-", "20D Low": "-"}

# ==============================================================================
# 4. MAIN CONTROLLER
# ==============================================================================
def run_scanner():
    print(f"\n{'='*60}")
    print(f"   INSTITUTIONAL SNIPER v5.0 (DEEP DATA)")
    print(f"{'='*60}\n")

    summary_report = []
    deep_dives = {}

    for ticker in Config.WATCHLIST:
        print(f"Scanning {ticker}...", end="\r")
        info, hist = fetch_data(ticker)
        if info is None: continue

        # 1. Run Analysis
        fund_status = analyze_fundamentals(info)
        action, reason = check_institutional_action(hist, ticker)
        metrics = get_detailed_metrics(hist)

        # 2. Store Summary
        row = {
            "Ticker": ticker,
            "Action": action,
            "Reason": reason,
            "52W High": metrics["52W High"],
            "20D High": metrics["20D High"],
            "20D Low": metrics["20D Low"]
        }
        summary_report.append(row)

        # 3. Prepare Deep Dive for ALL stocks (since we are testing)
        # In the future, we can restrict this to only "BUY" signals
        deep_df = hist[['Open', 'High', 'Low', 'Close', 'Volume']].tail(20).copy()
        deep_dives[ticker] = deep_df

        time.sleep(1) # Respect NSE limits

    # --- DISPLAY ---
    if summary_report:
        print("\n" + " " * 50)
        print("--- 📊 MARKET SUMMARY ---")
        df_summary = pd.DataFrame(summary_report)
        from IPython.display import display
        display(df_summary)

    # Show Deep Dive for one stock to test (e.g., first one)
    if deep_dives:
        target = Config.WATCHLIST[0] # Let's look at Reliance first
        print(f"\n🔎 DEEP DIVE: {target} (Last 20 Days)")
        display(deep_dives[target])

if __name__ == "__main__":
    run_scanner()


   INSTITUTIONAL SNIPER v5.0 (DEEP DATA)

Scanning NETWEB.NS...
                                                  
--- 📊 MARKET SUMMARY ---


,Ticker,Action,Reason,52W High,20D High,20D Low
0,NETWEB.NS,Wait,"Normal (Z:-0.2, Del:20.6%)",4479 (2025-10-08),3469 (2026-01-09),3012 (2026-01-02)



🔎 DEEP DIVE: NETWEB.NS (Last 20 Days)


,Open,High,Low,Close,Volume
Date,,,,,
2025-12-18 00:00:00+05:30,3163.199951,3211.399902,3110.000000,3120.600098,317313
2025-12-19 00:00:00+05:30,3134.600098,3271.899902,3134.600098,3250.899902,578563
2025-12-22 00:00:00+05:30,3260.000000,3336.899902,3237.000000,3276.100098,333230
2025-12-23 00:00:00+05:30,3266.100098,3267.500000,3215.000000,3232.300049,227733
2025-12-24 00:00:00+05:30,3232.000000,3250.000000,3180.000000,3193.199951,204739
2025-12-25 00:00:00+05:30,3193.199951,3193.199951,3193.199951,3193.199951,0
2025-12-26 00:00:00+05:30,3193.000000,3230.000000,3130.000000,3147.600098,241643
2025-12-29 00:00:00+05:30,3147.600098,3189.000000,3065.000000,3078.800049,298436
2025-12-30 00:00:00+05:30,3080.000000,3124.000000,3065.500000,3093.300049,266979


In [1]:
# ==============================================================================
# PROJECT: TECHNO-FUNDA INSTITUTIONAL SCANNER (v2.0)
# OBJECTIVE: Identify Safe Stocks with Institutional Buying Momentum
# AUTHOR: You & Gemini
# ==============================================================================

# ------------------------------------------------------------------------------
# TABLE OF CONTENTS
# ------------------------------------------------------------------------------
# 1. IMPORTS & SETUP ......... Library imports and environment checks
# 2. CONFIGURATION ........... Central control panel for all thresholds
# 3. DATA ENGINE ............. Handles yfinance and NSE connections
# 4. STRATEGY LAYERS ......... The 3-Step Filtering Logic
# 5. MAIN CONTROLLER ......... The execution loop and reporting
# ------------------------------------------------------------------------------
#




fatal: not a git repository (or any of the parent directories): .git



# Understaning notes




1. Debt-to-Equity < 1.5 (The "Mortgage" Analogy)
Think of this like buying a house.

Equity is your Down Payment (the money you actually own in the house).

Debt is the Mortgage (the money you borrowed from the bank).

If you put ₹10 Lakhs down and borrow ₹90 Lakhs, your ratio is 9.0. You are "highly leveraged." If the house price drops even a little, you are in trouble. If you put ₹50 Lakhs down and borrow ₹50 Lakhs, your ratio is 1.0. You are "safe."

The Rule (< 1.5): For every ₹1 of the shareholders' own money, the company should not owe more than ₹1.50 to the bank.

Why? In bad times (like a recession), companies with high debt go bankrupt because they must pay interest regardless of whether they made a profit. Low debt companies can survive the storm.

2. ROE > 10% (The "Savings Account" Analogy)
ROE (Return on Equity) asks: "For every ₹100 of shareholder money, how much profit did the management generate this year?"

Your Question: "I think this should be higher?"

The Answer: You are absolutely right! You are thinking like a "Growth Investor." Most top-tier investors look for 15% or 20%+.

Why use 10% then? 10% is just the "Safety Floor." It filters out the "garbage" (companies losing money or barely breaking even).

A savings account gives you ~7% risk-free.

If a risky company can't even generate 10%, it's not worth your time.

1. VOL_LOOKBACK = 20 (The Baseline)
This tells the computer how far back to look to define what is "normal" for this stock.

Layman Term: "The Monthly Average."

Why 20? There are roughly 20 trading days in a month.

Example: Imagine a coffee shop ☕. To know if today is unusually busy, you can't just look at yesterday (which might have been a holiday). You look at the average sales over the last 20 days to get a solid baseline of what "normal" looks like.

2. Z_SCORE_LIMIT = 2.0 (The Anomaly Detector)
This measures how "weird" or "extreme" today's volume is compared to that baseline.

Layman Term: "The Excitement Level."

The Math: A Z-Score of 0 is perfectly normal. A Z-Score of +2.0 means volume is significantly higher than usual (statistically rare).

Example:

Normal day: The coffee shop sells 100 cups.

Busy day (Z=1): They sell 120 cups. (Busy, but happens).

Crazy day (Z=2): They sell 160 cups. Something special is definitely happening! 😲

3. DELIVERY_LIMIT = 40.0 (The Commitment Check)
This checks what percentage of the shares traded were actually "taken home" (delivered to a demat account) versus just bought and sold rapidly (intraday trading).

Layman Term: "The Serious Buyers."

Why 40%? If people are just gambling for quick profits, they sell before the market closes (Delivery = 0%). If they are investing for the long term, they keep the shares (Delivery = 100%).

Example:

Low Delivery (20%): A crowd enters the coffee shop, looks around, takes selfies, and leaves without buying much. Lots of noise, no real business.

High Delivery (40%+): The crowd enters, and almost half of them sit down and order a full meal. They are committed customers. ✅